# BigQuery Setup for MIMIC-IV Access

This notebook guides you through setting up BigQuery API to access MIMIC-IV datasets for the ARDS research project.

## 1. Prerequisites

Before running this notebook, ensure you have:
1. Access to MIMIC-IV on PhysioNet
2. Linked your PhysioNet account to Google Cloud BigQuery
3. A Google Cloud Project with BigQuery API enabled

### Steps to link PhysioNet to BigQuery:
1. Go to https://physionet.org/content/mimiciv/
2. Sign the data use agreement
3. Link to BigQuery following PhysioNet instructions

## 2. Install Required Libraries

In [1]:
# !pip install google-cloud-bigquery pandas pyarrow db-dtypes

## 3. Authentication Setup

### Option A: Using Google Cloud CLI (Recommended for local development)

In [42]:
# First, authenticate via terminal:
# gcloud auth application-default login

# Then set your project ID
import os
os.environ['GOOGLE_CLOUD_PROJECT'] = 'sccm2025'  # Replace with your GCP project ID

## 4. Initialize BigQuery Client

In [43]:
from google.cloud import bigquery
import pandas as pd

# Initialize the client
client = bigquery.Client()

# Verify connection
print(f"Connected to project: {client.project}")

/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Connected to project: sccm2025


## 5. MIMIC-IV Dataset Configuration

In [44]:
# MIMIC-IV BigQuery dataset IDs
# MIMIC_PROJECT = "physionet-data"
# MIMIC_DATASET = "mimiciv_hosp"
# MIMIC_ICU_DATASET = "mimiciv_icu"
# MIMIC_NOTE_DATASET = "mimiciv_note"
client = bigquery.Client()

dataset_id = "sccm-discovery.eicu_crd_ii_v0_2_0"
tables = client.list_tables(dataset_id)

for table in tables:
    print(table.table_id)

/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


admissiondrug
admissiondx
allergy
apacheapsvar
apachepatientresults
apachepredvar
careplancareprovider
careplaneol
careplangeneral
careplangoal
careplaninfectiousdisease
customlab
diagnosis
hospital
infusiondrug
intakeoutput
lab
medication
microlab
nurseassessment
nursecare
nursecharting
pasthistory
patient
physicalexam
respiratorycare
respiratorycharting
treatment
vitalaperiodic
vitalperiodic


## 6. Test Connection with Sample Queries

In [45]:
query = """
SELECT *
FROM `sccm-discovery.eicu_crd_ii_v0_2_0.patient`
LIMIT 1000
"""

df = client.query(query).to_dataframe()
print(df.head())

/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   patientunitstayid  patienthealthsystemstayid  gender   age ethnicity  \
0           12345218                   10425393  Female    54      None   
1           12267112                   10355358  Female    81      None   
2           12349649                   10429309  Female    89      None   
3           12380841                   10457220  Female    54      None   
4           12237114                   10328459    Male  > 89      None   

   hospitalid   wardid apacheadmissiondx  admissionheight  hospitaladmityear  \
0     2000967  2002430              None              NaN               2017   
1     2000967  2002430              None              NaN               2017   
2     2000967  2002430              None              NaN               2017   
3     2000967  2002430              None              NaN               2017   
4     2000967  2002430              None              NaN               2017   

   ... unitstaytype admissionweight  dischargeweight unitdischargeye

## 7. Explore meds

In [46]:
from google.cloud import bigquery
client = bigquery.Client()

# First, let's identify the column names in the medication and infusiondrug tables
query_med_cols = """
SELECT column_name 
FROM `sccm-discovery.eicu_crd_ii_v0_2_0.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'medication'
ORDER BY ordinal_position;
"""

query_inf_cols = """
SELECT column_name
FROM `sccm-discovery.eicu_crd_ii_v0_2_0.INFORMATION_SCHEMA.COLUMNS` 
WHERE table_name = 'infusiondrug'
ORDER BY ordinal_position;
"""

med_cols = client.query(query_med_cols).to_dataframe()
inf_cols = client.query(query_inf_cols).to_dataframe()

print("Medication table columns:")
print(med_cols)
print("\nInfusiondrug table columns:")
print(inf_cols)




/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Medication table columns:
           column_name
0         medicationid
1    patientunitstayid
2        drugorderyear
3      drugorderoffset
4        drugstartyear
5      drugstartoffset
6      drugivadmixture
7   drugordercancelled
8             drugname
9        drughiclseqno
10              dosage
11          routeadmin
12           frequency
13         loadingdose
14                 prn
15        drugstopyear
16      drugstopoffset
17                 gtc

Infusiondrug table columns:
         column_name
0     infusiondrugid
1  patientunitstayid
2       infusionyear
3     infusionoffset
4           drugname
5           drugrate
6       infusionrate
7         drugamount
8      volumeoffluid
9      patientweight


In [ ]:
# ---- 1. adjust the meds once here -----------------------------------------
med_list  = ["cisatracurium", "vecuronium", "rocuronium",
             "atracurium", "pancuronium"]
pattern   = "|".join(med_list)             # → 'cisatracurium|vecuronium|…'

# ---- 2. single SQL that unions medication + infusiondrug -------------------
query = f"""
-- Neuromuscular-blocking agents ordered or infused (eICU-CRD v2.0)
WITH nmba AS (
  -- discrete doses (medication table)
  SELECT
    patientUnitStayID,
    drugName                          AS med_name,
    dosage                           AS dose_entered,
    routeAdmin                        AS dose_unit,
    drugStartOffset                   AS offset_min,
    'medication'                      AS source_table
  FROM   `sccm-discovery.eicu_crd_ii_v0_2_0.medication`
  WHERE  REGEXP_CONTAINS(LOWER(drugName), r'{pattern}')

  UNION ALL

  -- continuous infusions (infusiondrug table)
  SELECT
    patientUnitStayID,
    drugName                          AS med_name,
    infusionRate                      AS dose_entered,
    drugRate                          AS dose_unit,
    infusionOffset                    AS offset_min,
    'infusiondrug'                    AS source_table
  FROM   `sccm-discovery.eicu_crd_ii_v0_2_0.infusiondrug`
  WHERE  REGEXP_CONTAINS(LOWER(drugName), r'{pattern}')
)

-- return everything so you can filter downstream in pandas
SELECT *
FROM   nmba
ORDER  BY patientUnitStayID, offset_min;
"""

df = client.query(query).to_dataframe()
df.head()

## ICD codes

In [15]:
query = """
-- Get diagnosis codes from eICU-CRD v2.0
SELECT 
    *
FROM `sccm-discovery.eicu_crd_ii_v0_2_0.diagnosis`
ORDER BY patientUnitStayID, diagnosisOffset
LIMIT 10;
"""

# Execute query and display first few rows
df_dx = client.query(query).to_dataframe()
df_dx.head()

/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,diagnosisid,patientunitstayid,diagnosisyear,diagnosisoffset,diagnosisstring,activeupondischarge,icd9code,diagnosispriority
0,104227212,11227959,2020,477,neurologic|trauma - CNS|intracranial injury,False,removed: not cleared,Other
1,104223922,11227959,2020,4775,pulmonary|respiratory failure|acute respirator...,False,"518.81, J96.00",Other
2,104227184,11227959,2020,4775,neurologic|trauma - CNS|intracranial injury,False,removed: not cleared,Other
3,104227163,11227959,2020,6057,neurologic|trauma - CNS|intracranial injury,True,removed: not cleared,Other
4,104224563,11227959,2020,6057,pulmonary|respiratory failure|acute respirator...,True,"518.81, J96.00",Other


## 8. eICU-CRD ARDS Cohort Analysis with PEEP Criteria - STROBE Flow

This section implements the ARDS cohort identification from eICU-CRD v2.0 with BigQuery connectivity.
The analysis follows these inclusion/exclusion steps with numbers for STROBE diagram:

### STROBE Flow Criteria:
1. **Adult >18** 
2. **At least one ICU admission**
3. **PEEP >= 5 within 48 hours of hospital admission**
4. **S/F ratio < 315**
5. **No heart failure and not pregnant**

### Step 1: Load All Patients and Apply Adult Filter (≥18 years)

In [48]:
import pandas as pd
import numpy as np
from google.cloud import bigquery

# Initialize client
client = bigquery.Client()

print("=== ARDS COHORT WITH PEEP CRITERIA - STROBE FLOW ===")
print("NMB = Neuromuscular Blockade (paralytic drugs)")
print("Adding PEEP ≥ 5 cmH2O criteria to ARDS definition...")
print("Tracking numbers for STROBE diagram...")

# Define dataset path
DATASET_ID = "sccm-discovery.eicu_crd_ii_v0_2_0"

# Track STROBE numbers
strobe_numbers = {}

/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


=== ARDS COHORT WITH PEEP CRITERIA - STROBE FLOW ===
NMB = Neuromuscular Blockade (paralytic drugs)
Adding PEEP ≥ 5 cmH2O criteria to ARDS definition...
Tracking numbers for STROBE diagram...


### Step 2: Identify Exclusions (Pregnancy and Heart Failure)

In [49]:
query_patients = f"""
SELECT 
    *
FROM `{DATASET_ID}.patient`
LIMIT 10
"""

patient_df = client.query(query_patients).to_dataframe()
patient_df.dtypes

/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


patientunitstayid                          Int64
patienthealthsystemstayid                  Int64
gender                                    object
age                                       object
ethnicity                                 object
hospitalid                                 Int64
wardid                                     Int64
apacheadmissiondx                         object
admissionheight                          float64
hospitaladmityear                          Int64
hospitaladmittime24          datetime64[us, UTC]
hospitaladmittime                         object
hospitaladmitoffset                        Int64
hospitaladmitsource                       object
hospitaldischargeyear                      Int64
hospitaldischargetime24      datetime64[us, UTC]
hospitaldischargetime                     object
hospitaldischargeoffset                    Int64
hospitaldischargelocation                 object
hospitaldischargestatus                   object
unittype            

In [50]:
# Step 1: Total population and Adult patients (≥18 years)
print("\n--- STEP 1: LOADING ALL PATIENTS ---")

# First get total population
query_total = f"""
SELECT COUNT(*) as total_patients
FROM `{DATASET_ID}.patient`
"""

total_result = client.query(query_total).to_dataframe()
strobe_numbers['total_population'] = int(total_result['total_patients'].iloc[0])
print(f"Total population in database: {strobe_numbers['total_population']:,}")

# Then get adults
query_adults = f"""
WITH cleaned AS (
  SELECT 
    patientunitstayid,
    age,
    gender,
    ethnicity,
    unitdischargestatus,
    hospitaladmityear,
    unitdischargeyear,
    hospitaldischargeyear,
    admissionheight,
    admissionweight,
    CASE 
        WHEN REGEXP_CONTAINS(age, r'^[0-9]+$') THEN CAST(age AS FLOAT64)
        WHEN REGEXP_CONTAINS(age, r'([0-9]+)') THEN CAST(REGEXP_EXTRACT(age, r'([0-9]+)') AS FLOAT64)
        ELSE NULL 
    END AS age_numeric
  FROM `{DATASET_ID}.patient`
)
SELECT *
FROM cleaned
WHERE age_numeric >= 18
"""

patient_df = client.query(query_adults).to_dataframe()
strobe_numbers['adult_patients'] = len(patient_df)
print(f"Adult patients (≥18 years): {strobe_numbers['adult_patients']:,}")

# Convert age to numeric for further analysis
patient_df['age_numeric'] = pd.to_numeric(patient_df['age'], errors='coerce')
patient_df = patient_df[patient_df['age_numeric'] >= 18].copy()
strobe_numbers['valid_adults'] = len(patient_df)
print(f"Valid adult patients: {strobe_numbers['valid_adults']:,}")


--- STEP 1: LOADING ALL PATIENTS ---


/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Total population in database: 201,727
Adult patients (≥18 years): 201,537
Valid adult patients: 194,826


### Step 3: Identify ARDS Patients by ICD Codes and Diagnosis Strings

In [51]:
# Step 2: Exclusions - Pregnancy and Heart Failure
print("\n--- STEP 2: IDENTIFYING EXCLUSIONS ---")

# Get diagnosis data for exclusions
query_exclusions = f"""
SELECT 
    patientunitstayid,
    icd9code,
    diagnosisstring
FROM `{DATASET_ID}.diagnosis`
WHERE 
    -- Pregnancy codes
    (REGEXP_CONTAINS(icd9code, r'V22|V23|V24|630|631|632|633|634|635|636|637|638|639|640|641|642|643|644|645|646|647|648|649|650|651|652|653|654|655|656|657|658|659')
    OR REGEXP_CONTAINS(LOWER(diagnosisstring), r'pregnan|gravid|maternity|obstetric|delivery|labor|gestation'))
    OR
    -- Heart failure codes  
    (REGEXP_CONTAINS(icd9code, r'398.91|402.01|402.11|402.91|404.01|404.03|404.11|404.13|404.91|404.93|428|I50|I11.0|I13.0|I13.2')
    OR REGEXP_CONTAINS(LOWER(diagnosisstring), r'heart failure|cardiac failure|congestive heart|CHF|systolic.*failure|diastolic.*failure'))
"""

exclusion_df = client.query(query_exclusions).to_dataframe()

# Identify pregnant patients
pregnant_patients = exclusion_df[
    exclusion_df['icd9code'].str.contains(
        'V22|V23|V24|630|631|632|633|634|635|636|637|638|639|640|641|642|643|644|645|646|647|648|649|650|651|652|653|654|655|656|657|658|659', 
        na=False, regex=True
    ) |
    exclusion_df['diagnosisstring'].str.contains(
        'pregnan|gravid|maternity|obstetric|delivery|labor|gestation', 
        case=False, na=False, regex=True
    )
]['patientunitstayid'].unique()

# Identify heart failure patients
heart_failure_patients = exclusion_df[
    exclusion_df['icd9code'].str.contains(
        '398.91|402.01|402.11|402.91|404.01|404.03|404.11|404.13|404.91|404.93|428|I50|I11.0|I13.0|I13.2', 
        na=False, regex=True
    ) |
    exclusion_df['diagnosisstring'].str.contains(
        'heart failure|cardiac failure|congestive heart|CHF|systolic.*failure|diastolic.*failure', 
        case=False, na=False, regex=True
    )
]['patientunitstayid'].unique()

excluded_patients = set(pregnant_patients) | set(heart_failure_patients)
strobe_numbers['pregnant_patients'] = len(pregnant_patients)
strobe_numbers['heart_failure_patients'] = len(heart_failure_patients)
strobe_numbers['total_excluded'] = len(excluded_patients)

print(f"Excluded patients: {strobe_numbers['total_excluded']:,}")
print(f"  - Pregnant: {strobe_numbers['pregnant_patients']:,}")
print(f"  - Heart failure: {strobe_numbers['heart_failure_patients']:,}")


--- STEP 2: IDENTIFYING EXCLUSIONS ---
Excluded patients: 8,472
  - Pregnant: 303
  - Heart failure: 8,178


### Step 4: PEEP ≥ 5 cmH2O within 48 hours of admission

In [52]:
# Step 3: ARDS by ICD/diagnosis
print("\n--- STEP 3: IDENTIFYING ARDS BY ICD/DIAGNOSIS ---")

query_ards_dx = f"""
SELECT DISTINCT
    patientunitstayid
FROM `{DATASET_ID}.diagnosis`
WHERE 
    icd9code = '518.82'
    OR REGEXP_CONTAINS(LOWER(diagnosisstring), r'ARDS|acute respiratory distress|respiratory failure')
"""

ards_diagnosis_df = client.query(query_ards_dx).to_dataframe()
ards_icd_patients = ards_diagnosis_df['patientunitstayid'].unique()
strobe_numbers['ards_by_icd'] = len(ards_icd_patients)

print(f"ARDS by ICD/diagnosis: {strobe_numbers['ards_by_icd']:,} patients")


--- STEP 3: IDENTIFYING ARDS BY ICD/DIAGNOSIS ---
ARDS by ICD/diagnosis: 24,002 patients


### Step 5: S/F Ratio ≤ 315 Criteria

In [53]:
query_peep = f"""
SELECT 
    *
FROM `{DATASET_ID}.respiratorycharting`
LIMIT 10
"""

peep_df = client.query(query_peep).to_dataframe()

In [54]:
peep_df.dtypes

respchartid              Int64
patientunitstayid        Int64
respchartyear            Int64
respchartoffset          Int64
respchartentryyear       Int64
respchartentryoffset     Int64
respcharttypecat        object
respchartvaluelabel     object
respchartvalue          object
dtype: object

In [55]:
# Step 4: PEEP ≥ 5 cmH2O criteria within 48 hours
print("\n--- STEP 4: PEEP ≥ 5 cmH2O CRITERIA ---")

# Get PEEP values for all patients in first 48 hours (2880 minutes)
query_peep = f"""
SELECT 
    patientunitstayid,
    respchartoffset,
    SAFE_CAST(respchartvalue AS FLOAT64) AS peep_numeric,
    respchartvaluelabel
FROM `{DATASET_ID}.respiratorycharting`
WHERE 
    respchartvaluelabel = 'PEEP'
    AND respchartoffset >= 0 
    AND respchartoffset <= 2880  -- First 48 hours
    AND SAFE_CAST(respchartvalue AS FLOAT64) IS NOT NULL
    AND SAFE_CAST(respchartvalue AS FLOAT64) BETWEEN 0 AND 30
"""

peep_df = client.query(query_peep).to_dataframe()
strobe_numbers['peep_measurements'] = len(peep_df)
print(f"PEEP measurements in first 48h: {strobe_numbers['peep_measurements']:,}")

# Calculate patient-level PEEP statistics
peep_df_clean = peep_df.dropna(subset=['peep_numeric']).copy()
strobe_numbers['valid_peep_measurements'] = len(peep_df_clean)
print(f"Valid PEEP measurements: {strobe_numbers['valid_peep_measurements']:,}")

# Find patients with PEEP ≥ 5
peep_summary = peep_df_clean.groupby('patientunitstayid').agg({
    'peep_numeric': ['max', 'mean', lambda x: (x >= 5).any()]
}).reset_index()

peep_summary.columns = ['patientunitstayid', 'max_peep', 'mean_peep', 'peep_ge_5']
peep_eligible_patients = peep_summary[peep_summary['peep_ge_5'] == True]['patientunitstayid'].tolist()
strobe_numbers['peep_eligible'] = len(peep_eligible_patients)

print(f"Patients with PEEP ≥ 5 cmH2O: {strobe_numbers['peep_eligible']:,}")
print(f"PEEP statistics: Mean {peep_summary['mean_peep'].mean():.1f}, Max {peep_summary['max_peep'].max():.1f}")


--- STEP 4: PEEP ≥ 5 cmH2O CRITERIA ---


/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


PEEP measurements in first 48h: 690,871
Valid PEEP measurements: 690,871
Patients with PEEP ≥ 5 cmH2O: 46,172
PEEP statistics: Mean 6.9, Max 30.0


In [56]:
# Step 5: S/F ratio ≤ 315 criteria (within 48 hours)
print("\n--- STEP 5: S/F RATIO ≤ 315 CRITERIA ---")

# Get FiO2 and SpO2 data efficiently with BigQuery
query_resp_data = f"""
WITH resp_data AS (
  SELECT 
    patientunitstayid,
    respchartoffset,
    SAFE_CAST(respchartvalue AS FLOAT64) AS resp_value,
    respchartvaluelabel
  FROM `{DATASET_ID}.respiratorycharting`
  WHERE 
    respchartvaluelabel IN ('FiO2', 'SpO2', 'O2 Sat', 'SaO2')
    AND respchartoffset BETWEEN 0 AND 2880  -- First 48 hours
),
fio2_data AS (
  SELECT *
  FROM resp_data 
  WHERE respchartvaluelabel = 'FiO2'
    AND resp_value BETWEEN 21 AND 100
),
spo2_data AS (
  SELECT *
  FROM resp_data
  WHERE respchartvaluelabel IN ('SpO2', 'O2 Sat', 'SaO2')
    AND resp_value BETWEEN 70 AND 100
)

-- Calculate S/F ratios by matching closest FiO2 and SpO2 within 60 minutes
SELECT 
  f.patientunitstayid,
  f.respchartoffset AS fio2_offset,
  f.resp_value AS fio2_value,
  s.respchartoffset AS spo2_offset, 
  s.resp_value AS spo2_value,
  (s.resp_value / f.resp_value * 100) AS sf_ratio
FROM fio2_data f
JOIN spo2_data s 
  ON f.patientunitstayid = s.patientunitstayid
  AND ABS(s.respchartoffset - f.respchartoffset) <= 60
"""

sf_df = client.query(query_resp_data).to_dataframe()
strobe_numbers['sf_calculations'] = len(sf_df)
print(f"S/F ratio calculations: {strobe_numbers['sf_calculations']:,}")

# Find patients with S/F ≤ 315
ards_sf_patients = sf_df[sf_df['sf_ratio'] <= 315]['patientunitstayid'].unique()
strobe_numbers['ards_by_sf'] = len(ards_sf_patients)
print(f"Patients with S/F ≤ 315: {strobe_numbers['ards_by_sf']:,}")


--- STEP 5: S/F RATIO ≤ 315 CRITERIA ---


/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


S/F ratio calculations: 437,803
Patients with S/F ≤ 315: 14,251


In [57]:
# Step 6: FINAL ARDS COHORT with ALL CRITERIA
print("\n--- STEP 6: FINAL ARDS COHORT (ALL CRITERIA) ---")

# Combine all ARDS criteria
ards_by_diagnosis = set(ards_icd_patients)
ards_by_sf = set(ards_sf_patients)
ards_by_peep = set(peep_eligible_patients)

print("ARDS criteria overlap:")
print(f"- ICD/Diagnosis: {len(ards_by_diagnosis):,} patients")
print(f"- S/F ratio ≤ 315: {len(ards_by_sf):,} patients")  
print(f"- PEEP ≥ 5 cmH2O: {len(ards_by_peep):,} patients")

# Patients meeting ANY ARDS criteria (ICD OR S/F)
ards_any_criteria = ards_by_diagnosis | ards_by_sf
strobe_numbers['ards_any_criteria'] = len(ards_any_criteria)
print(f"- ANY ARDS criteria: {strobe_numbers['ards_any_criteria']:,} patients")

# Patients meeting ARDS + PEEP criteria
ards_with_peep = ards_any_criteria & ards_by_peep
strobe_numbers['ards_with_peep'] = len(ards_with_peep)
print(f"- ARDS + PEEP ≥ 5: {strobe_numbers['ards_with_peep']:,} patients")

# Apply adult and exclusion filters
adult_patient_ids = set(patient_df['patientunitstayid'])
final_cohort_ids = (ards_with_peep & adult_patient_ids) - excluded_patients
strobe_numbers['final_cohort'] = len(final_cohort_ids)

print(f"\nFINAL ARDS COHORT: {strobe_numbers['final_cohort']:,} patients")
print("✅ Adult (≥18 years)")
print("✅ At least one ICU admission (inherent in eICU)")
print("✅ ARDS (ICD codes OR S/F ≤ 315)")  
print("✅ PEEP ≥ 5 cmH2O within 48h")
print("✅ No pregnancy/heart failure")

# Track excluded by step
strobe_numbers['excluded_age'] = strobe_numbers['total_population'] - strobe_numbers['valid_adults']
strobe_numbers['excluded_no_ards'] = strobe_numbers['valid_adults'] - strobe_numbers['ards_any_criteria']
strobe_numbers['excluded_no_peep'] = strobe_numbers['ards_any_criteria'] - strobe_numbers['ards_with_peep']
strobe_numbers['excluded_conditions'] = strobe_numbers['total_excluded']

print(f"\n=== STROBE DIAGRAM NUMBERS ===")
print(f"Total population: {strobe_numbers['total_population']:,}")
print(f"Excluded <18 years: {strobe_numbers['excluded_age']:,}")
print(f"Adult population: {strobe_numbers['valid_adults']:,}")
print(f"Excluded no ARDS criteria: {strobe_numbers['excluded_no_ards']:,}")
print(f"With ARDS criteria: {strobe_numbers['ards_any_criteria']:,}")
print(f"Excluded no PEEP ≥5: {strobe_numbers['excluded_no_peep']:,}")
print(f"With ARDS + PEEP: {strobe_numbers['ards_with_peep']:,}")
print(f"Excluded pregnancy/HF: {strobe_numbers['excluded_conditions']:,}")
print(f"FINAL COHORT: {strobe_numbers['final_cohort']:,}")


--- STEP 6: FINAL ARDS COHORT (ALL CRITERIA) ---
ARDS criteria overlap:
- ICD/Diagnosis: 24,002 patients
- S/F ratio ≤ 315: 14,251 patients
- PEEP ≥ 5 cmH2O: 46,172 patients
- ANY ARDS criteria: 32,989 patients
- ARDS + PEEP ≥ 5: 18,658 patients

FINAL ARDS COHORT: 16,345 patients
✅ Adult (≥18 years)
✅ At least one ICU admission (inherent in eICU)
✅ ARDS (ICD codes OR S/F ≤ 315)
✅ PEEP ≥ 5 cmH2O within 48h
✅ No pregnancy/heart failure

=== STROBE DIAGRAM NUMBERS ===
Total population: 201,727
Excluded <18 years: 6,901
Adult population: 194,826
Excluded no ARDS criteria: 161,837
With ARDS criteria: 32,989
Excluded no PEEP ≥5: 14,331
With ARDS + PEEP: 18,658
Excluded pregnancy/HF: 8,472
FINAL COHORT: 16,345


In [58]:
# Step 7: EXPOSURES ANALYSIS WITH REFINED COHORT
print("\n=== STEP 7: EXPOSURES IN REFINED COHORT ===")

# Convert final_cohort_ids to list for BigQuery
final_cohort_list = list(final_cohort_ids)
final_cohort_list = [int(x) for x in final_cohort_ids]

# Proning search using BigQuery
query_prone = f"""
SELECT DISTINCT patientunitstayid, cplitemvalue
FROM `{DATASET_ID}.careplangeneral`
WHERE patientunitstayid IN UNNEST(@cohort_ids)
  AND REGEXP_CONTAINS(LOWER(cplitemvalue), r'prone|proning|position|turn|reposit|Turn/ROM')
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("cohort_ids", "INT64", final_cohort_list)
    ]
)

prone_df = client.query(query_prone, job_config=job_config).to_dataframe()
prone_patients = set(prone_df['patientunitstayid'].unique()) if len(prone_df) > 0 else set()
strobe_numbers['prone_patients'] = len(prone_patients)

# NMB search using BigQuery (from provided drug list)
nmb_drugs = ['cisatracurium', 'vecuronium', 'rocuronium', 'atracurium', 'succinylcholine', 'pancuronium']
nmb_pattern = '|'.join(nmb_drugs)

query_nmb = f"""
-- Neuromuscular blockade from medication table
SELECT DISTINCT patientunitstayid, drugname, 'medication' as source
FROM `{DATASET_ID}.medication`
WHERE patientunitstayid IN UNNEST(@cohort_ids)
  AND REGEXP_CONTAINS(LOWER(drugname), r'{nmb_pattern}')

UNION ALL

-- Neuromuscular blockade from infusiondrug table  
SELECT DISTINCT patientunitstayid, drugname, 'infusiondrug' as source
FROM `{DATASET_ID}.infusiondrug`
WHERE patientunitstayid IN UNNEST(@cohort_ids)
  AND REGEXP_CONTAINS(LOWER(drugname), r'{nmb_pattern}')
"""

nmb_df = client.query(query_nmb, job_config=job_config).to_dataframe()
nmb_patients = set(nmb_df['patientunitstayid'].unique()) if len(nmb_df) > 0 else set()
strobe_numbers['nmb_patients'] = len(nmb_patients)

print(f"NMB patients: {strobe_numbers['nmb_patients']}/{strobe_numbers['final_cohort']} ({strobe_numbers['nmb_patients']/strobe_numbers['final_cohort']*100:.1f}%)")
print(f"Proning patients: {strobe_numbers['prone_patients']}/{strobe_numbers['final_cohort']} ({strobe_numbers['prone_patients']/strobe_numbers['final_cohort']*100:.1f}%)")

if len(nmb_df) > 0:
    print(f"\nTop NMB drugs in refined cohort:")
    print(nmb_df['drugname'].value_counts().head(10))

if len(prone_df) > 0:
    print(f"\nProning entries found: {len(prone_df)} patients")


=== STEP 7: EXPOSURES IN REFINED COHORT ===
NMB patients: 3741/16345 (22.9%)
Proning patients: 782/16345 (4.8%)

Top NMB drugs in refined cohort:
drugname
rocuronium                           2087
succinylcholine                      1313
ROCURONIUM BROMIDE                    385
cisatracurium (mcg/kg/min)            258
cisatracurium                         252
Cisatracurium (mcg/kg/min)             90
SUCCINYLCHOLINE                        49
Atracurium (mcg/kg/min)                48
Cisatracurium IVF Infused (ml/hr)      44
Rocuronium (mcg/kg/min)                33
Name: count, dtype: int64

Proning entries found: 782 patients


In [59]:
# Step 8: OUTCOMES ANALYSIS AND RESEARCH FEASIBILITY
print("\n=== STEP 8: OUTCOMES ANALYSIS AND RESEARCH FEASIBILITY ===")

# Get outcomes for final cohort
final_cohort_df = patient_df[patient_df['patientunitstayid'].isin(final_cohort_ids)].copy()
mortality = (final_cohort_df['unitdischargestatus'] == 'Expired').sum()
strobe_numbers['mortality'] = mortality
strobe_numbers['mortality_rate'] = mortality/len(final_cohort_df)*100

print(f"Cohort outcomes:")
print(f"- ICU mortality: {strobe_numbers['mortality']}/{strobe_numbers['final_cohort']} ({strobe_numbers['mortality_rate']:.1f}%)")
print(f"- Mean age: {final_cohort_df['age_numeric'].mean():.1f} years")

# Research feasibility analysis
total_exposures = strobe_numbers['nmb_patients'] + strobe_numbers['prone_patients']
strobe_numbers['total_exposures'] = total_exposures

print(f"\n=== RESEARCH FEASIBILITY WITH PEEP CRITERIA ===")
print(f"Total patients with exposures: {strobe_numbers['total_exposures']}")

if total_exposures >= 10:
    feasibility = "✅ Potentially feasible for descriptive analysis"
elif total_exposures >= 5:
    feasibility = "⚠️  Limited but possible case series"
else:
    feasibility = "❌ Too few exposures for meaningful analysis"

print(feasibility)

print(f"\nRecommendation based on {strobe_numbers['final_cohort']:,} ARDS patients:")
if strobe_numbers['final_cohort'] >= 100:
    cohort_assessment = "✅ Good ARDS cohort size for analysis"
elif strobe_numbers['final_cohort'] >= 50:
    cohort_assessment = "⚠️  Moderate cohort size"
else:
    cohort_assessment = "❌ Small cohort - consider expanding criteria"

print(cohort_assessment)

print(f"\n=== NEXT STEPS ===")
print("1. Review refined cohort characteristics")
print("2. Extract timing variables for exposures")
print("3. Prepare outcome analysis with available sample")
print("4. Consider whether PEEP criteria improves study quality")

print(f"\n=== FINAL COHORT SUMMARY FOR STROBE ===")
print(f"Total ARDS patients: {strobe_numbers['final_cohort']:,}")
print(f"With NMB exposure: {strobe_numbers['nmb_patients']}")
print(f"With proning exposure: {strobe_numbers['prone_patients']}")
print(f"ICU mortality rate: {strobe_numbers['mortality_rate']:.1f}%")

# Export STROBE numbers for diagram creation
print(f"\n=== COMPLETE STROBE FLOW NUMBERS ===")
for key, value in strobe_numbers.items():
    print(f"{key}: {value:,}" if isinstance(value, int) else f"{key}: {value:.1f}" if isinstance(value, float) else f"{key}: {value}")

# Save final cohort for further analysis
final_cohort_df_export = final_cohort_df.copy()
strobe_numbers['cohort_exported'] = True


=== STEP 8: OUTCOMES ANALYSIS AND RESEARCH FEASIBILITY ===
Cohort outcomes:
- ICU mortality: 3939/16345 (24.1%)
- Mean age: 62.4 years

=== RESEARCH FEASIBILITY WITH PEEP CRITERIA ===
Total patients with exposures: 4523
✅ Potentially feasible for descriptive analysis

Recommendation based on 16,345 ARDS patients:
✅ Good ARDS cohort size for analysis

=== NEXT STEPS ===
1. Review refined cohort characteristics
2. Extract timing variables for exposures
3. Prepare outcome analysis with available sample
4. Consider whether PEEP criteria improves study quality

=== FINAL COHORT SUMMARY FOR STROBE ===
Total ARDS patients: 16,345
With NMB exposure: 3741
With proning exposure: 782
ICU mortality rate: 24.1%

=== COMPLETE STROBE FLOW NUMBERS ===
total_population: 201,727
adult_patients: 201,537
valid_adults: 194,826
pregnant_patients: 303
heart_failure_patients: 8,178
total_excluded: 8,472
ards_by_icd: 24,002
peep_measurements: 690,871
valid_peep_measurements: 690,871
peep_eligible: 46,172
sf_c